In [1]:
import argparse
from datetime import datetime, date
from uti import DataLoader, Logger
from Database import GSDatabase, GSPriceDf
import re
from Model.settings import DataCleaner
import pandas as pd
import numpy as np
from datetime import datetime, date

GSD = GSDatabase()
logger = Logger()
DL = DataLoader()
DC = DataCleaner()


\
Current root directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital
Current Base directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi
Current Database directory: C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database


2022-05-11 13:16:31,202 P[18424] [MainThread 26080] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2022-05-11 13:16:31,204 P[18424] [MainThread 26080] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-05-11 13:16:31,204 P[18424] [MainThread 26080] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2022-05-11 13:16:31,205 P[18424] [MainThread 26080] Port number was not identified, cannot send any request


Current working directory C:\Users\JayYang\Platform\Citi
13:16:32 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Log/20220511/20220511131631.txt created.
13:16:32 Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0
13:16:32 Engine current settings: 
daily_position 18 
region_position 6 
skewness: 0


In [ ]:
"""
1. Get train data before march, used as the scoring training data on reports after march

2. Pull the reports after march, 
- Apply before march scoring on march reports
- Apply before april scoring on april reports
- Apply scoring for Goldman and Citi

3. From "Jay pnl for benchmark.csv", calculate the number of longs/shorts on each day; real trades pnl, times ATR(x) to reach the 
pnl as is the case where ATR(x)=1 is used in system calculation

4. Select trades from Goldman/Citi based on daily numbers of shorts/longs
Three cases to consider
- Wholly Goldman
- Wholly Citi
- Randomly from Goldman/Citi

"""

In [2]:
jay_pnl = DL.loadDB('Jay pnl for benchmark.csv', parse_dates=['Entry Date', 'Exit Date'])

13:16:32 C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Log/20220511/20220511131632.txt created.
13:16:32 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Jay pnl for benchmark.csv


In [3]:
jay_pnl

,Ticker,ATR,Reason,Side,ATR(x),ATR Used,Risk,Position,Position (R),Entry Price,Exit Price,Entry Date,Exit Date,Realized R(old),Realized gains,Realized R
0,SAN SM Equity,0.1200,(GS) AH+,long,2.0,0.2400,45.179362,188.247342,188.247342,3.178,3.100,2022-03-31,2022-03-31,-0.325000,-14.683293,-0.650000
1,AAL LN Equity,149.0000,(GS) EC+,long,2.0,298.0000,3805.754301,12.770988,12.770988,3994.000,3972.500,2022-03-31,2022-03-31,-0.072148,-274.576233,-0.144295
2,URW NA Equity,2.9651,(GS) AH+,long,2.0,5.9302,45.179362,7.618522,7.618522,69.000,68.180,2022-03-31,2022-03-31,-0.138275,-6.247188,-0.276551
3,DSM NA Equity,4.6036,(Citi) ER-,short,2.0,9.2072,45.179362,4.906960,4.906960,163.650,162.500,2022-03-31,2022-03-31,0.124902,5.643004,0.249805
4,SGRE SM Equity,0.9608,(Citi) TPC-,short,2.0,1.9216,45.179362,23.511325,23.511325,16.230,15.975,2022-03-31,2022-03-31,0.132702,5.995388,0.265404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,9531 JP Equity,49.7000,(Mizuho) ER +,long,2.0,99.4000,13083.867590,131.628447,131.628447,2310.000,2463.000,2022-04-28,2022-04-28,1.539235,20139.152330,3.078471
258,ASSAB SS Equity,7.6360,(Citi) TPC+,long,2.0,15.2720,987.303280,64.647936,64.647936,253.700,244.100,2022-04-28,2022-04-28,-0.628601,-620.620186,-1.257203
259,AVV LN Equity,131.6000,(Citi) TPC-,short,2.0,263.2000,8027.614996,30.500057,30.500057,1885.500,2054.000,2022-04-28,2022-04-28,-0.640198,-5139.259600,-1.280395
260,KNIN SW Equity,1.4552,(Citi) TPC-,short,2.0,2.9104,97.191175,33.394439,33.394439,45.130,45.230,2022-04-28,2022-04-28,-0.034360,-3.339444,-0.068719


In [4]:
train_data, test_data = DC.get_benchmark_test_data()

13:16:32 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Backtest/Benchmark data/X_train.csv
13:16:33 Loading C:/Users/JayYang/OneDrive - Alpha Sherpa Capital/Citi/Database\Backtest/Benchmark data/X_test.csv


In [5]:
all_data = pd.concat([train_data, test_data], axis=0)

In [18]:
from Model.rules import benchmark_rule, benchmark_rule2, benchmark_rule3
from Model.benchmark import get_daily_trade
from Model.LR import LR

In [ ]:
# Get Citi Reports with scores for March

In [13]:
train_data_before_mar = all_data[all_data['Date'] < datetime(2022, 3, 1)].copy()
test_data_mar = all_data[(all_data['Date'] >= datetime(2022, 3, 1)) & (all_data['Date'] < datetime(2022, 4, 1))].copy()
test_data_mar

,Time,Summary,Date,Ticker,Tickers,company,Headline,Region,Industry,Head analyst,...,release_period,ticker_updated,gap_in_atr,exch_region2,market_cap_grp,side,stop_loss,d0_r,d1_r,d2_r
12574,2022-03-01 14:54:00,"Macau's Feb-22 GGR came in at MOP7,759m (or ~M...",2022-03-01,0027.HK,"0027.HK,LVS,0200.HK,MLCO,2282.HK,MGM,1928.HK,0...",Galaxy Entertainment,"Macau Gaming: Feb-22 GGR MOP7,759m; ~31% of Fe...",HK,Casinos & Gaming,"George Choi, CFA",...,Within,0027.HK HK,-0.023185,Asia,LargeCap,short,45.556536,-0.510077,-0.301409,-1.298378
12575,2022-03-01 15:46:00,We expect Veeva to deliver a solid close to FY...,2022-03-01,VEEV,VEEV,Veeva Systems Inc,Veeva Systems Inc (VEEV.N): Looking to bounce ...,US,Application Software,Tyler Radke,...,Before,VEEV US,-0.040546,Americas,LargeCap,short,237.228514,-0.500072,-0.362021,3.254328
12576,2022-03-01 15:36:00,Gamuda's consortium has been awarded the contr...,2022-03-01,GAMU.KL,GAMU.KL,Gamuda,Gamuda (GAMU.KL): Second win for the year: A$2...,MK,Engineering & Construction,Wai Hoong Liew,...,Within,GAMU.KL MK,0.257959,Asia,SmallCap,long,2.812468,2.708566,4.256318,4.514276
12577,2022-03-01 15:29:00,The #1 question we get asked by investors on t...,2022-03-01,LLOY.L,"LLOY.L,UBSG.S,SOGN.PA,ABNd.AS,DBKGn.DE,SWEDa.ST",Lloyds Banking Group PLC,WATCH: European Diversified Banks and Regional...,LN,Diversified Banks,"Andrew Coombs, CFA",...,Before,LLOY.L LN,-0.783266,Europe,LargeCap,short,51.212465,1.021409,0.911640,1.253970
12578,2022-03-01 15:27:00,"Q4 loss/FCF burn better than our model, but th...",2022-03-01,LCID,LCID,Lucid Group Inc,Lucid Group Inc (LCID.O): Q4 Takeaways; Loweri...,US,Automobile Manufacturers,Itay Michaeli,...,Before,LCID US,0.153082,Americas,LargeCap,short,27.032653,-0.198222,-0.037289,0.728121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14240,2022-03-31 07:49:00,We dialed into DFM’s FY21 post-results analyst...,2022-03-31,0489.HK,0489.HK,Dongfeng Motor,Dongfeng Motor (0489.HK): FY21 Post-earnings B...,HK,Automobile Manufacturers,Jeff Chung,...,Before,0489.HK HK,-0.093326,Asia,MidCap,short,6.314304,0.979918,0.746604,0.139988
14241,2022-03-31 07:53:00,Wacker unveiled their 2030 strategy with a hea...,2022-03-31,WCHG.DE,WCHG.DE,Wacker Chemie AG,Wacker Chemie AG (WCHG.DE): 2030 Strategy Anno...,GY,Industrial Gases,"Mubasher Chaudhry, ACA",...,Before,WCHG.DE GY,-0.020927,Europe,MidCap,short,159.228552,-0.062781,-1.000000,-1.000000
14242,2022-03-31 08:05:00,Prosus hosted a deep dive call on Quick Commer...,2022-03-31,PRX.AS,"PRX.AS,NPNJn.J",Prosus N.V,Prosus N.V (PRX.AS): Quick Commerce Deep Dive;...,NaN,Interactive Media & Services,Catherine T O'Neill,...,Before,PRX.AS NA,0.089858,Europe,LargeCap,long,47.335405,-0.515707,0.377013,1.334195
14243,2022-03-31 08:22:00,We are introducing a new methodology for sizin...,2022-03-31,EQIX,EQIX,Equinix Inc.,Equinix Inc. (EQIX.O): A Potentially Better Wa...,US,Integrated Telecommunication Services,"Michael Rollins, CFA",...,After,EQIX US,-0.116717,Americas,LargeCap,short,778.451234,0.806229,-0.242261,-0.313860


In [19]:
test_data_mar['side'] = benchmark_rule(test_data_mar)
test_data_mar['side'] = benchmark_rule2(test_data_mar)
test_data_mar['side'] = benchmark_rule3(test_data_mar)
train_data_before_mar, test_data_mar, elements_name = get_daily_trade(train_data_before_mar, test_data_mar)  # Calculate factor scores

model = LR(train_data_before_mar[elements_name], train_data_before_mar[['d0_r']])
model.train()
model.evaluate()
intercept, ols_weights = model.get_params()

# Scoring: in Americas use OLS, in Asia and Europe use simple average.
us_mask = (test_data_mar['exch_region'] == 'Americas')
test_data_mar['ols_score'] = np.sum(ols_weights * test_data_mar[elements_name], axis=1) + intercept[0]
test_data_mar['mean_score'] = np.mean(test_data_mar[elements_name], axis=1)
test_data_mar['score'] = np.where(us_mask, test_data_mar['ols_score'], test_data_mar['mean_score'])
test_data_mar.loc[test_data_mar['score'] <= 0, 'side'] = ''
test_data_mar.loc[test_data_mar['score'] <= 0, 'score'] = 0
test_data_mar.loc[~test_data_mar['side'].isin(['long', 'short']), 'score'] = 0

test_data_mar['d0_exp'] = np.where(us_mask, test_data_mar['ols_score'], test_data_mar['mean_score'])

Head analyst_score
['Head analyst_score']
Ticker_score
['Head analyst_score', 'Ticker_score']
release_period_score
['Head analyst_score', 'Ticker_score', 'release_period_score']
Report Type_score
['Head analyst_score', 'Ticker_score', 'release_period_score', 'Report Type_score']
Intercept: [-0.07894801]
Coefficients: [[0.20056853 0.97149182 0.7914407  0.7432175 ]]
Score:  0.055885939678842256
R2 score:  0.055885939678842256
MSE score:  0.6927523518254863
RMSE score:  0.8323174585610267
                            OLS Regression Results                            
Dep. Variable:                   d0_r   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     870.0
Date:                Wed, 11 May 2022   Prob (F-statistic):               0.00
Time:                        16:31:47   Log-Likelihood:                -72639.
No. Observations:               587

C:\Users\JayYang\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# Get Citi Reports with scores for April

In [14]:
train_data_before_apr = all_data[all_data['Date'] < datetime(2022, 4, 1)].copy()
test_data_apr = all_data[(all_data['Date'] >= datetime(2022, 4, 1)) & (all_data['Date'] < datetime(2022, 5, 1))].copy()
test_data_apr

,Time,Summary,Date,Ticker,Tickers,company,Headline,Region,Industry,Head analyst,...,release_period,ticker_updated,gap_in_atr,exch_region2,market_cap_grp,side,stop_loss,d0_r,d1_r,d2_r
14245,2022-04-01 16:12:00,Adapting to new business model for high-qualit...,2022-04-01,1233.HK,1233.HK,Times China,Times China (1233.HK): Still Poised to Bring a...,HK,Diversified Real Estate Activities,Cindy Li,...,After,1233.HK HK,0.244029,Asia,SmallCap,long,2.572170,0.732086,1.464172,0.823597
14246,2022-04-01 14:22:00,Vestas' announced orders through March 31st to...,2022-04-01,VWS.CO,VWS.CO,Vestas,"Vestas (VWS.CO): Orders ahead, but policy stil...",DC,Electrical Components & Equipment,Martin Wilkie,...,Before,VWS.CO DC,0.021085,Europe,LargeCap,short,214.485538,-0.532389,0.495491,-1.000000
14247,2022-04-01 16:00:00,Focus on Freight – Conversation With a FedEx ISP,2022-04-01,CNI,"CNI,CP,CSX,FDX,UNP",Canadian National Railway Company,"Wetherbee's Weekly Freight Line: Friday, April...",US,Logistics - Surface,Christian Wetherbee,...,Before,CNI US,-0.201850,Americas,LargeCap,short,136.635711,2.361265,1.481504,1.957565
14248,2022-04-01 15:57:00,We hosted an investor call with Haier Smart Ho...,2022-04-01,6690.HK,6690.HK,Haier Smart Home,Haier Smart Home (6690.HK): Citi Call Takeaway...,HK,Household Products,"Mark Li, CFA",...,Within,6690.HK HK,0.434612,Asia,LargeCap,short,26.150452,-1.000000,-1.000000,-1.000000
14249,2022-04-01 14:59:00,"Sodexo has delivered strong 1H22 results, with...",2022-04-01,EXHO.PA,EXHO.PA,Sodexo,Sodexo (EXHO.PA): Strong 1H22 results; expect ...,FP,Restaurants,James Ainley,...,Before,EXHO.PA FP,0.070218,Europe,LargeCap,long,69.281392,-1.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15379,2022-04-29 00:52:00,We hosted BBMG's 1Q22 post-results conference ...,2022-04-29,2009.HK,"2009.HK,601992.SS",BBMG,BBMG (2009.HK): Transcript: 1Q22 Post-Results ...,HK,Engineering & Construction,"Jack Shang, CFA",...,Before,2009.HK HK,-0.238891,Asia,MidCap,short,1.291860,-0.716672,0.000000,0.000000
15380,2022-04-29 00:45:00,"1Q22 net profit of ENN Natural Gas (""ENN NG"") ...",2022-04-29,600803.SS,600803.SS,ENN Natural Gas,ENN Natural Gas (600803.SS): Strong Profit Gro...,CH,Gas Utilities,"Pierre Lau, CFA",...,Before,600803.SS CH,-0.179640,Asia,MidCap,long,15.330663,0.359280,0.000000,0.000000
15381,2022-04-29 00:43:00,"Baosteel reported its 2021 NI at Rmb23.6bn, up...",2022-04-29,600019.SS,600019.SS,Baoshan Iron & Steel,Baoshan Iron &amp; Steel (600019.SS): 1Q22 NI ...,CH,Steel,"Jack Shang, CFA",...,Before,600019.SS CH,-0.189596,Asia,LargeCap,short,6.493719,-0.872141,0.000000,0.000000
15382,2022-04-29 00:20:00,"COSL net profit came in at Rmb304m, up 68% yoy...",2022-04-29,2883.HK,2883.HK,China Oilfield Services,China Oilfield Services (2883.HK): Earnings Se...,HK,Oil & Gas Equipment & Services,Toby Shek,...,Before,2883.HK HK,0.185009,Asia,MidCap,long,7.381639,0.766464,0.000000,0.000000


In [20]:
test_data_apr['side'] = benchmark_rule(test_data_apr)
test_data_apr['side'] = benchmark_rule2(test_data_apr)
test_data_apr['side'] = benchmark_rule3(test_data_apr)
train_data_before_apr, test_data_apr, elements_name = get_daily_trade(train_data_before_apr, test_data_apr)  # Calculate factor scores

model = LR(train_data_before_apr[elements_name], train_data_before_apr[['d0_r']])
model.train()
model.evaluate()
intercept, ols_weights = model.get_params()

# Scoring: in Americas use OLS, in Asia and Europe use simple average.
us_mask = (test_data_apr['exch_region'] == 'Americas')
test_data_apr['ols_score'] = np.sum(ols_weights * test_data_apr[elements_name], axis=1) + intercept[0]
test_data_apr['mean_score'] = np.mean(test_data_apr[elements_name], axis=1)
test_data_apr['score'] = np.where(us_mask, test_data_apr['ols_score'], test_data_apr['mean_score'])
test_data_apr.loc[test_data_apr['score'] <= 0, 'side'] = ''
test_data_apr.loc[test_data_apr['score'] <= 0, 'score'] = 0
test_data_apr.loc[~test_data_apr['side'].isin(['long', 'short']), 'score'] = 0

test_data_apr['d0_exp'] = np.where(us_mask, test_data_apr['ols_score'], test_data_apr['mean_score'])

Head analyst_score
['Head analyst_score']
Ticker_score
['Head analyst_score', 'Ticker_score']
release_period_score
['Head analyst_score', 'Ticker_score', 'release_period_score']
Report Type_score
['Head analyst_score', 'Ticker_score', 'release_period_score', 'Report Type_score']
Intercept: [-0.07798255]
Coefficients: [[0.20695757 0.97168663 0.77927487 0.74564423]]
Score:  0.056358280043373954
R2 score:  0.056358280043373954
MSE score:  0.687632301487559
RMSE score:  0.8292359745498015
                            OLS Regression Results                            
Dep. Variable:                   d0_r   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     902.8
Date:                Wed, 11 May 2022   Prob (F-statistic):               0.00
Time:                        16:34:59   Log-Likelihood:                -74479.
No. Observations:               6046

C:\Users\JayYang\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [21]:
results = pd.concat([test_data_mar, test_data_apr], axis=0)

In [24]:
results['Broker'] = 'Citi'

In [25]:
DL.toDB(results, 'Citi system reports.csv')